<a href="https://colab.research.google.com/github/kjg331/Finetune-of-self-introduction-evaluation-language-model-/blob/main/Finetune_self_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Set up Kernel, Load Required Dependencies, Dataset and LLM

## 1-1. Set up Kernel and Required Dependencies

In [1]:
!pip install evaluate==0.4.0
!pip install rouge_score==0.1.2
!pip install -q datasets accelerate loralib huggingface_hub
!pip install -q git+https://github.com/huggingface/peft.git git+https://github.com/huggingface/transformers.git
!pip install -q -U trl
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 36.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=20184bf7d3939800841e677

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import tempfile
import logging
import random
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import json

from pprint import pprint
from datasets import Dataset

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import TrainingArguments
from transformers import Trainer
from transformers import PreTrainedTokenizerFast
from transformers import BitsAndBytesConfig
from accelerate.utils import load_and_quantize_model, BnbQuantizationConfig
from accelerate import Accelerator

from trl import SFTTrainer

logger = logging.getLogger(__name__)
global_config = None

# 1-2. Load Dataset and LLM
I used the dataset from Jobkorea[https://www.jobkorea.co.kr/]<br>
and LLm called "EleutherAI/polyglot-ko-1.3b"[https://huggingface.co/EleutherAI/polyglot-ko-1.3b]

In [5]:
columns = ['position', 'question', 'answer', 'advice']
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/dataset_for_letter/content.csv", names = columns, header=None, encoding='UTF-8')

In [13]:
df['input'] = df['question'] + "\n"+ df['answer']
df = df.drop(["question", 'answer'], axis = 1)
df.head()

,position,advice,input
0,사무담당자,"아쉬운점 1 사례에 대한 배경 설명을 쓴 부분인데, 내용이 부족합니다. 어디에서 이...","(최근 5년 내에) 귀하가 어떤 구체적인 목표를 달성하기 위해, 어떤 준비와 과정을..."
1,사무담당자,"아쉬운점 1 사례에 대한 배경 설명을 쓴 부분인데, 내용이 부족합니다. 어디에서 이...","(최근 5년 내에) 비용이나 효율의 증대에 관련한 이슈나 문제에 대하여, 다른 사람..."
2,사무담당자,"아쉬운점 1 솔직한 지원동기인 것 같습니다. 좀 더 나은 환경, 좀 더 금융다운 일...","귀하가 농협에 지원한 구체적인 동기에 대하여 제시하고, 그러한 지원동기가 농협에게 ..."
3,법인영업,좋은점 1 내용에 있어 특별함은 없지만 항목의 요구 사항에 맞게 작성하려고 노력한 ...,"본인 성격의 장단점 및 가치관을 성장과정(가족, 학창시절, 주변인과의 관계)에 기반..."
4,법인영업,아쉬운점 1 문제가 되는 것은 아니지만 인터넷을 활용하면 쉽게 찾을 수 있는 회사 ...,신한은행 지원 동기 및 해당 분야에 지원한 이유를 구체적으로 작성해 주세요. (최대...


In [8]:
# Load the base model
model_name = 'EleutherAI/polyglot-ko-1.3b'

# Create a configuration for quantization
# https://huggingface.co/blog/4bit-transformers-bitsandbytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Enable 4-bit quantization for weights and activations
    bnb_4bit_quant_type="nf4",  # Set specific quantization type
    bnb_4bit_compute_dtype=torch.float16,  # Use half-precision float16 for computations
)

# Load the pre-trained model and apply quantization configuration
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name, device_map='auto', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
original_model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', quantization_config=bnb_config, trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/748M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [14]:
device_count = torch.cuda.device_count()
if device_count > 0:
    #logger.debug("Select GPU device")
    print("Select GPU device")
    device = torch.device("cuda")
else:
    #logger.debug("Select CPU device")
    print("Select CPU device")
    device = torch.device("cpu")

Select GPU device


In [15]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 123408384
all model parameters: 727830528
percentage of trainable model parameters: 16.96%


### 1.3 - Test the Model with Zero Shot Inferencing

Test the model with the zero shot inferencing. You can see that the model struggles to answer the instruction compared to the baseline advcie, that it doesn't have any relavant response. Let's see if it can make a better response after fine-tune with 1000 examples.

In [16]:
def question_prompot(example, index):
    prompt=f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][index]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][index]}

    ### advice:
    """
    return prompt

# Define function to carry out output from model(GPU)
def zero_gen(example, model, index):
    prompt=f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][index]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][index]}
    """
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=400,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    response = tokenizer.decode(gened[0])
    return response[len(prompt):]

In [17]:
index = 3

advice = df['advice'][index]
prompt = question_prompot(df, index)

zero_output = zero_gen(df, original_model, index)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ADVICE:\n{advice}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{zero_output}')

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:447: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     법인영업 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    본인 성격의 장단점 및 가치관을 성장과정(가족, 학창시절, 주변인과의 관계)에 기반하여 작성해 주세요. (최대 1,500자 입력가능)
"신뢰를 만든 커뮤니케이션, 플래너를 통해 배운 계획의 중요성"

제 성격의 장점은 다양한 사람과 자연스럽게 어울리는 커뮤니케이션 역량입니다.
어릴 적 어머니의 문구점을 도운 경험과 학창시절의 임원 활동으로, 저는 친구들뿐만 아니라 어른들과도 자연스럽게 어울리는 커뮤니케이션 역량을 쌓았습니다. 그리고 수많은 팀 프로젝트를 진행하며, 선후배 팀원과 빠르게 유대감을 쌓고 팀의 분위기를 만들며 커뮤니케이션 역량을 발전시켰습니다. 그 결과, 저는 팀원에게 먼저 인정받는 리더의 신뢰뿐만 아니라 팀원들과의 인간적인 신뢰도 쌓을 수 있었습니다.

한편, 저는 많은 일이 집중되었을 때 집중력이 약해지곤 합니다. 일례로, 저는 대학 시절, 전공 공부와 학회 활동, 아르바이트를 동시에 진행하며 어려움을 겪었습니다. 그때 플래너를 사용하여 일주일 단위로 계획을 세웠고, 업무를 완수할 수 있었습니다. 이후 저는 계획을 세우는 습관을 키웠고, 효율적인 시간 관리로 다양한 업무 성과를 만들 수 있었습니다.좋은점 1

"스스로 기회를 만드는 사람, 130개 거절 속에서 타결해 낸 공연 후원 가능성"

저는 원하는 목표를 이루기 위해 

# 2 - Perform Parameter Efficient Fine-Tuning (PEFT)

Now, let's perform **Parameter Efficient Fine-Tuning (PEFT)** fine-tuning as opposed to "full fine-tuning" as you did above. PEFT is a form of instruction fine-tuning that is much more efficient than full fine-tuning - with comparable evaluation results as you will see soon.

PEFT is a generic term that includes **Low-Rank Adaptation (LoRA)** and prompt tuning (which is NOT THE SAME as prompt engineering!). In most cases, when someone says PEFT, they typically mean LoRA. LoRA, at a very high level, allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU). After fine-tuning for a specific task, use case, or tenant with LoRA, the result is that the original LLM remains unchanged and a newly-trained “LoRA adapter” emerges. This LoRA adapter is much, much smaller than the original LLM - on the order of a single-digit % of the original LLM size (MBs vs GBs).  

That said, at inference time, the LoRA adapter needs to be reunited and combined with its original LLM to serve the inference request.  The benefit, however, is that many LoRA adapters can re-use the original LLM which reduces overall memory requirements when serving multiple tasks and use cases.

### 2.1 - Preprocess the Dialog-Summary Dataset

You need to convert the content-advice (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with `Give the adivice for the following self-introduce` and to the start of the advcie with `Advice` as follows:

Training prompt (dialogue):
```
### Instruction:
"Below is an instruction that describes a task, paired with an input that provides further context. \
Write a response that appropriately completes the request.
{position} 직무의 자기 소개서에 대해 평가해주세요.

### context:
{input}

### Answer:"""
```

Training response (advcie):
```
아쉬운점 1 사례에 대한 배경 설명을 쓴 부분인데, 내용이 부족합니다. 어디에서 이 경험을 한 것이고 지원자의 신분이 무엇이었는지, 그리고 왜 이 업무를 했는지에 대한 설명이 제시되어야 경험을 가치를 제대로 판단할 수 있습니다. 그리고 5년 이내라는 구체적인 기간을 제시하고 있으므로 5년 이내의 경험이라는 것을 알 수 있도록 구체적인 시기를 제시할 필요가 있습니다.
아쉬운점 2 이 부분의 문제는 내용이 쉽게 이해되지 않는다는 것입니다. 지원자는 제안서를 쓰는 업무였는데, 업무 초기 담당 실무자는 어떤 사람을 말하는 것인지, 취급 비중이 큰 상품의 실무자는 또 무슨 말인 것인지 등 제시한 내용이 쉽게 읽히지 않습니다. 자기소개서는 지원자의 경험에 대해 전혀 모르는 사람이 읽는 글이므로 이 부분을 유의하여 작성해야 합니다.
```


Then preprocess the prompt-response dataset into tokens and pull out their `input_ids` (1 per token).

In [19]:
def question_prompot(example, index):
    prompt=f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][index]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][index]}

    ### advice:
    """
    return prompt


# Definition of a function that tokenizes the given input
def formatting_prompts_func(example):
    text =f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position']} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input']}

    ### Advice:
    {example['advice']}
    """
    return text

In [20]:
example_dataset = df[:1000]

dataset = Dataset.from_pandas(example_dataset)
dataset

Dataset({
    features: ['position', 'advice', 'input'],
    num_rows: 1000
})

In [21]:
split_dataset = dataset.train_test_split(test_size=0.2, shuffle=True, seed=123)
print(split_dataset)

DatasetDict({
    train: Dataset({
        features: ['position', 'advice', 'input'],
        num_rows: 800
    })
    test: Dataset({
        features: ['position', 'advice', 'input'],
        num_rows: 200
    })
})


In [22]:
print(f"Shapes of the datasets:")
print(f"Training: {split_dataset['train'].shape}")
print(f"Test: {split_dataset['test'].shape}")

print(split_dataset)

Shapes of the datasets:
Training: (800, 3)
Test: (200, 3)
DatasetDict({
    train: Dataset({
        features: ['position', 'advice', 'input'],
        num_rows: 800
    })
    test: Dataset({
        features: ['position', 'advice', 'input'],
        num_rows: 200
    })
})


### 2.2 - Setup the PEFT/LoRA model for Fine-Tuning

You need to set up the PEFT/LoRA model for fine-tuning with a new layer/parameter adapter. Using PEFT/LoRA, you are freezing the underlying LLM and only training the adapter. Have a look at the LoRA configuration below. Note the rank (`r`) hyper-parameter, which defines the rank/dimension of the adapter to be trained.[https://huggingface.co/docs/peft/en/quicktour]

- r: The rank of the low-rank decomposition, which influences the number of adaptable parameters.
- lora_alpha: A scaling factor determining the initial magnitude of the LORA parameters.
- target_modules: The neural network components we wish to reparameterize. Here, we're targeting the "q" (query) and "v" (value) modules in the transformer's attention mechanism.
- lora_dropout:  A regularising dropout is applied to the LORA parameters to prevent overfitting.
- bias: Specifies the nature of the bias term in the reparameterization. Setting it to "none" means no bias will be added.
- task_type: Signifying the type of task for which we're employing LORA. In our case, it's sequence-to-sequence language modelling, perfectly aligned with our Flan-T5's capabilities.

In [23]:
from peft import LoraConfig, get_peft_model

# Define hyperparameters for LoRa (Low-Rank Attention) regularization
lora_alpha = 16  # Controls the rank of the low-rank attention matrices
lora_dropout = 0.1  # Dropout rate applied to the LoRa layers
lora_r = 64  # Rank of the low-rank attention matrices

# Create a LoraConfig object with specified parameters
lora_config = LoraConfig(
    lora_alpha=lora_alpha,  # Set the hyperparameter values
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",  # Set bias option (none, learned, shared)
    task_type="CAUSAL_LM",  # Specify the task type (e.g., CAUSAL_LM, MASKED_LM)
    target_modules=[  # List of modules to apply LoRa regularization
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ],
)

In [24]:
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 50331648
all model parameters: 778162176
percentage of trainable model parameters: 6.47%


### 3.2 - Train PEFT Adapter

Define training arguments and create `Trainer` instance.

In [25]:
# Define the output directory for training results
output_dir = f'/content/drive/MyDrive/Colab Notebooks/llm_projects/job_prepare/peft-self-eval-training-{str(int(time.time()))}'

# Set the training batch size per device (e.g., GPU)
per_device_train_batch_size = 8

# Accumulate gradients over multiple batches before optimization step
gradient_accumulation_steps = 4

# Choose the optimizer (here, "paged_adamw_32bit")
optim = "paged_adamw_32bit"

# Save model checkpoints and evaluation results every `save_steps` steps
save_steps = 50

# Log training information every `logging_steps` steps
logging_steps = 10

# Set the initial learning rate for the optimizer
learning_rate = 2e-4

# Clip gradients to prevent exploding gradients
max_grad_norm = 0.3

# Maximum number of training steps
max_steps = 500

# Warm-up period as a ratio of total training steps where learning rate gradually increases
warmup_ratio = 0.03

# Use a constant learning rate scheduler (no decay)
lr_scheduler_type = "constant"

# Create a TrainingArguments object with specified parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,  # Enable mixed-precision training using half-precision
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,  # Group training samples by similar sequence lengths
    lr_scheduler_type=lr_scheduler_type,
)

In [26]:
# Set the maximum sequence length for input and output
max_seq_length = 300

# Create an SFTTrainer object for fine-tuning
# https://huggingface.co/docs/trl/en/sft_trainer
peft_trainer = SFTTrainer(
    model=original_model,                   # The language model to be fine-tuned
    train_dataset=split_dataset["train"],  # The training dataset
    eval_dataset=split_dataset["test"],   # The evaluation dataset
    peft_config=lora_config,             # LoRa regularization configuration
    #dataset_text_field="advice",       # The field in the dataset containing text
    max_seq_length=max_seq_length,     # Maximum sequence length for training
    tokenizer=tokenizer,               # The tokenizer for text pre-processing
    args=training_arguments,          # Training arguments defined previously
    formatting_func=formatting_prompts_func,
    packing=True
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:310: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [27]:
start = time.time()
peft_trainer.train()

peft_model_path="/content/drive/MyDrive/Colab Notebooks/llm_projects/job_prepare/peft-self-advice-checkpoint-local"
end=time.time()
time_taken=end-start
print(time_taken)

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

Step,Training Loss
10,3.058800
20,2.525900
30,2.164000
40,2.042800
50,1.982600
60,1.950400
70,1.937300
80,1.909400
90,1.898200
100,1.861900


2886.721488237381


('/content/drive/MyDrive/Colab Notebooks/llm_projects/job_prepare/peft-self-advice-checkpoint-local/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/llm_projects/job_prepare/peft-self-advice-checkpoint-local/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/llm_projects/job_prepare/peft-self-advice-checkpoint-local/tokenizer.json')

In [34]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto', trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       peft_model_path,
                                       torch_dtype=torch.bfloat16,
                                       quantization_config=bnb_config,
                                       is_trainable=False)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [41]:
tokenizer.pad_token = tokenizer.eos_token

In [35]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 1382141952
percentage of trainable model parameters: 0.00%



### 2.3 - Evaluate the Model Qualitatively (Human Evaluation)

As with many GenAI applications, a qualitative approach where you ask yourself the question "Is my model behaving the way it is supposed to?" is usually a good starting point. In the example below, you can see how the fine-tuned model is able to create a reasonable advice for the self-introductino compared to the original inability to understand what is being asked of the model. In addtion you can see how the result is changed with the one-shot prompt compare to the zero-shot prompt.

In [ ]:
peft_model.to(device)

In [67]:
def question_prompot(example, index):
    prompt=f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][index]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][index]}

    ### advice:
    """
    return prompt

# Define function to carry out output from model(GPU)
def zero_gen(example, model, index):
    prompt=f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][index]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][index]}

    ### advice:
    """
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=400,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)
    return response[len(prompt):]

def one_shot_gen(example, model, index):
    prompt=f"""
    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][0]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][0]}

    ### advice:
    {example['advice'][0]}

    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     {example['position'][index]} 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    {example['input'][index]}

    ### advice:
    """
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=400,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        pad_token_id=tokenizer.eos_token_id
    )
    response = tokenizer.decode(gened[0], skip_special_tokens=True)
    return response[len(prompt):]

In [68]:
index = 3

advice = df['advice'][index]
prompt = question_prompot(df, index)

zero_output = zero_gen(df, peft_model, index)
one_output = one_shot_gen(df, peft_model, index)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN ADVICE:\n{advice}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{zero_output}')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{one_output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    ### Instruction:
    "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
     법인영업 직무의 자기 소개서에 대해 평가해주세요.

    ### context:
    본인 성격의 장단점 및 가치관을 성장과정(가족, 학창시절, 주변인과의 관계)에 기반하여 작성해 주세요. (최대 1,500자 입력가능)
"신뢰를 만든 커뮤니케이션, 플래너를 통해 배운 계획의 중요성"

제 성격의 장점은 다양한 사람과 자연스럽게 어울리는 커뮤니케이션 역량입니다.
어릴 적 어머니의 문구점을 도운 경험과 학창시절의 임원 활동으로, 저는 친구들뿐만 아니라 어른들과도 자연스럽게 어울리는 커뮤니케이션 역량을 쌓았습니다. 그리고 수많은 팀 프로젝트를 진행하며, 선후배 팀원과 빠르게 유대감을 쌓고 팀의 분위기를 만들며 커뮤니케이션 역량을 발전시켰습니다. 그 결과, 저는 팀원에게 먼저 인정받는 리더의 신뢰뿐만 아니라 팀원들과의 인간적인 신뢰도 쌓을 수 있었습니다.

한편, 저는 많은 일이 집중되었을 때 집중력이 약해지곤 합니다. 일례로, 저는 대학 시절, 전공 공부와 학회 활동, 아르바이트를 동시에 진행하며 어려움을 겪었습니다. 그때 플래너를 사용하여 일주일 단위로 계획을 세웠고, 업무를 완수할 수 있었습니다. 이후 저는 계획을 세우는 습관을 키웠고, 효율적인 시간 관리로 다양한 업무 성과를 만들 수 있었습니다.좋은점 1

"스스로 기회를 만드는 사람, 130개 거절 속에서 타결해 낸 공연 후원 가능성"

저는 원하는 목표를 이루기 위해 

In [69]:
pprint(f'MODEL GENERATION - ONE SHOT:\n{zero_output}')

('MODEL GENERATION - ONE SHOT:\n'
 ' 아쉬운점 1 질문의 요구사항을 반영하지 않은 무작정 추천서라는 느낌이 듭니다. 이 내용으로 인해 지원자가 왜 130개 거절을 극복할 수 '
 '있었는지에 대한 내용이 전혀 없다는 점이 아쉽습니다. 다만, 문제해결과정에서 지원자가 어떻게 리더십을 발휘하였는지를 제시한 점이 '
 '좋습니다.\n'
 '    ')


In [70]:
pprint(f'MODEL GENERATION - ONE SHOT:\n{one_output}')

('MODEL GENERATION - ONE SHOT:\n'
 ' 아쉬운점 1 문제 해결을 위해 저는 어떠한 노력을 하였고 그러한 노력을 통해서 어떠한 성과를 얻었는지에 대한 설명이 제시되어야 합니다. '
 '또한 제시된 목표가 성취될 확률이 상대적으로 어느 정도이며 그것이 얼마나 가능한지에 대해서도 함께 검토할 필요가 있습니다. 그리고 '
 '목표했던 일을 성공적으로 완수하기 위해, 어떠한 준비 과정을 거쳐 이루어냈는지를 상세하게 설명해야 합니다.\n'
 '    ')


After the fine-tune with only 1,000 examples we can find the response getting far better than before which was

ORIGINAL MODEL GENERATION - ZERO SHOT:
##### context:형용사 형(-y)      "Yes(아니다)" 라는 대답은 NO(아니라)로 대답하셨네요?(예, 아니요 / 아닙니다)       형저는 어렸을 때부터 영어를 좋아하여 영어로 책을 읽고 영어로 일기를 쓰는 것을 생활화하였습니다. 또한, 대학 재학 기간 동안 많은 활동을 통하여 영어가 제 미래의 경쟁력이라는 생각을 하였습니다.형저는 해외 경험이 없었습니다. 하지만 제가 생각한 바를 전달할 수 있었습니다. 이를 통해 저는 학생들의 도전과 실패에서 얻는 소중한 경험을 알게 되었습니다. 좋은 경험이라고 생각합니다. 형저는 영어가 잘 표현되는 글이나 문서를 읽고, 생각을 정리하는 습관이 제게 있었습니다. 이와 같이 제가 배운 것들을 잘 전달할 수 있었습니다. 형저는 국내 대학 활동과 외국어인 영어, 그리고 사회 경험들을 다양한 방법으로 실천할 것입니다. 형저는 대학생으로서 할 수 있는 것이 많다고 생각합니다. 형저는 스스로 도전하기 위해, 국내 대학의 다양한 활동에 동참한다면 자신의 발전에 도움이 될 것이라고 생각합니다.형저는 영어를 활용한 대외활동에 많은 노력을 기울이겠습니다. 형저는 자신의 생각과 의견을 영어와 한국어로도 전달할 수 있는 준비가 갖춰져 있습니다. 형저는 외국어로서 영어와 한국어 모두 잘할 수 있는 능력이 갖추어져 있습니다. 형저는 자신의 지식과 정보를 다른 사람과 교류할 수 있는 기회가 있다고 생각합니다. 형저는 영어 뿐 만 아니라 한국어, 그리고 외국어로도 다른 사람과의 소통을 위해 최선을 다할 것입니다.형저